# Cell Neighborhood Diversity
This notebook can be used to determine the homogeneity/diversity of the neighbors surrounding each of the cells in our images.

In [9]:
import os
import pandas as pd
from functools import reduce   

import ark.settings as settings
from ark.analysis.neighborhood_analysis import compute_neighborhood_diversity

### Path Setup
* `spatial_analysis_dir`: the path to the directory containing the spatial analysis output
* `neighbors_mat_dir`: the path to directory containing the neighborhood matrix for your data
* `diversity_dir`: path for a new directory that will be created for the output below

In [28]:
base_dir = '../data/example_dataset'

spatial_analysis_dir = os.path.join(base_dir, "spatial_analysis")
neighbors_mat_dir = os.path.join(spatial_analysis_dir, "neighborhood_mats")
diversity_dir = os.path.join(spatial_analysis_dir, "cell_neighborhood_diversity")

if not os.path.exists(diversity_dir):
    os.makedirs(diversity_dir)

### Shannon Diversity
The code below will calculate the Shannon Diversity Index for each cell input. You must specify which neighbors matrix should be used based on the pixel radius and cell type column. Provide multiple cell cluster columns and the diversity will be calculated for each, then combined into a single results table.
- `pixel_radius`: radius used to define the neighbors of each cell
- `cell_type_columns`: list of columns in your cell table containing different classifications of cell phenotypes

The results will be saved to `neighborhood_diversity_radius50.csv` in the new cell neighborhood diversity directory.

In [ ]:
pixel_radius = 50
cell_type_columns = [settings.CELL_TYPE, ]

In [30]:
diversity_data = []
for cell_type_col in cell_type_columns:
    freqs_path = os.path.join(neighbors_mat_dir, f"neighborhood_freqs-{cell_type_col}_radius{pixel_radius}.csv")
    neighbor_freqs = pd.read_csv(freqs_path) 
    diversity_data.append(compute_neighborhood_diversity(neighbor_freqs, cell_type_col))

all_diversity_data = reduce(lambda left, right: 
                            pd.merge(left , right, on=[settings.FOV_ID, settings.CELL_LABEL]), diversity_data)
all_diversity_data.to_csv(os.path.join(diversity_dir, f'neighborhood_diversity_radius{pixel_radius}.csv'), index=False)